In [6]:
import pandas as pd

In [7]:
# dor = pd.read_excel(r"C:\Users\O304312\OneDrive - Kaiser Permanente\Documents\DOR Transactions.xlsx")
# dor = dor[dor["Program Area"] != "KPOCT"]
# dor = dor[["Project ID", "Total Cash Receipts"]].copy()

# vp = pd.read_excel(r"C:\Users\O304312\Downloads\Transaction.xlsx")
# vp = vp[["Service Line Code", "Transaction Amount"]].copy()
# vp["Transaction Amount"] = pd.to_numeric(vp["Transaction Amount"], errors="coerce").fillna(0)
# vp = vp.dropna(subset=["Service Line Code"])
# vp = vp.groupby("Service Line Code", as_index=False)["Transaction Amount"].sum().reset_index(drop=False)

In [8]:
# import pandas as pd
# from datetime import datetime


# # User-configurable date range
# START_DATE = "2000-01-01"  # Format: YYYY-MM-DD
# END_DATE = "2025-12-31"  # Format: YYYY-MM-DD

# start_date = pd.to_datetime(START_DATE)
# end_date = pd.to_datetime(END_DATE)

# print(f"Filtering data from {START_DATE} to {END_DATE}\n")

# dor = pd.read_excel(
#     r"C:\Users\O304312\OneDrive - Kaiser Permanente\Documents\DOR Transactions.xlsx"
# )
# # dor = dor[dor["Program Area"] != "KPOCT"]
# # dor = dor[dor["Program Area"] != "Pedi-Onc"]
# # dor = dor[dor["Funder Type"] == "INDUSTRY"]


# dor["Award Term Start Date"] = pd.to_datetime(
#     dor["Award Term Start Date"], errors="raise"
# )


# if "Award Term Start Date" in dor.columns:
#     dor["Award Term Start Date"] = pd.to_datetime(dor["Award Term Start Date"])
#     dor = dor[
#         (dor["Award Term Start Date"] >= start_date)
#         & (dor["Award Term Start Date"] <= end_date)
#     ]
#     print(f"DOR rows after date filter: {len(dor)}")

# # Ensure Project Title exists, then keep it
# if "Project Title" not in dor.columns:
#     dor["Project Title"] = pd.NA
# dor = dor[["Project ID", "Project Title", "Total Cash Receipts"]].copy()

# # Ensure Program Area exists, then keep it in DOR for merge/filters
# if "Program Area" not in dor.columns:
#     dor["Program Area"] = pd.NA
# dor = dor[["Project ID", "Project Title", "Total Cash Receipts", "Program Area"]].copy()

# vp = pd.read_excel(r"C:\Users\O304312\Downloads\Transaction.xlsx")
# if "Unnamed: 0" in vp.columns:
#     vp = vp.drop(columns=["Unnamed: 0"])

# vp_dup_count = vp.duplicated().sum()
# print(f"Number of duplicate rows in VP before drop_duplicates: {vp_dup_count}")

# vp = vp.drop_duplicates()

# # Filter VP by date (adjust column name if needed - e.g., "Accountable Completed Date")
# if "Accountable Completed Date" in vp.columns:
#     vp["Accountable Completed Date"] = pd.to_datetime(
#         vp["Accountable Completed Date"], errors="coerce"
#     )
#     vp = vp[
#         (vp["Accountable Completed Date"] >= start_date)
#         & (vp["Accountable Completed Date"] <= end_date)
#     ]
#     print(f"VP rows after date filter: {len(vp)}")

# # Ensure Site Study Code exists, then include it in the aggregation
# if "Site Study Code" not in vp.columns:
#     vp["Site Study Code"] = pd.NA

# vp = vp[["Service Line Code", "Site Study Code", "Transaction Amount"]].copy()
# vp["Transaction Amount"] = pd.to_numeric(
#     vp["Transaction Amount"], errors="coerce"
# ).fillna(0)
# vp = vp.dropna(subset=["Service Line Code"])

# # Aggregate amounts per Service Line Code and collect unique Site Study Codes
# vp = vp.groupby("Service Line Code", as_index=False).agg(
#     {
#         "Transaction Amount": "sum",
#         "Site Study Code": lambda s: (
#             "; ".join(sorted(set(map(str, s.dropna())))) if s.notna().any() else pd.NA
#         ),
#     }
# )

# # Merge on Project ID / Service Line Code
# merged = dor.merge(
#     vp, how="outer", left_on="Project ID", right_on="Service Line Code", indicator=True
# )
# merged["Total Cash Receipts"] = merged["Total Cash Receipts"].fillna(0)
# merged["Transaction Amount"] = merged["Transaction Amount"].fillna(0)

# # Get lists of IDs with no match (exclude DOR-only KPOCT/Pedi-Onc)
# dor_only_mask = (merged["_merge"] == "left_only") & (
#     ~merged["Program Area"].isin(["KPOCT", "Pedi-Onc"])
# )
# dor_only_ids = merged.loc[dor_only_mask, "Project ID"].dropna().tolist()
# vp_only_ids = (
#     merged.loc[merged["_merge"] == "right_only", "Service Line Code"].dropna().tolist()
# )

# print(
#     f"\n{len(dor_only_ids)} Project IDs in DOR but not in VP (excluding KPOCT/Pedi-Onc):"
# )
# print(dor_only_ids)

# print(f"\n{len(vp_only_ids)} Service Line Codes in VP but not in DOR:")
# print(vp_only_ids)

# # Export to CSV (include requested columns)
# merged.loc[
#     dor_only_mask, ["Project ID", "Project Title", "Total Cash Receipts"]
# ].to_csv("dor_only_ids.csv", index=False)
# merged.loc[
#     merged["_merge"] == "right_only",
#     ["Service Line Code", "Site Study Code", "Transaction Amount"],
# ].to_csv("vp_only_ids.csv", index=False)
# print("\nExported 'dor_only_ids.csv' and 'vp_only_ids.csv' for detailed review.")

# # Calculate summaries per your rule:
# # - Include KPOCT when it matches VP
# # - Exclude DOR-only KPOCT/Pedi-Onc from DOR totals
# matched_mask = merged["_merge"] == "both"
# overlap = merged.loc[matched_mask, "Total Cash Receipts"].sum()

# dor_only_total = merged.loc[dor_only_mask, "Total Cash Receipts"].sum()
# total_dor_effective = overlap + dor_only_total

# total_vp = vp["Transaction Amount"].sum()

# print(
#     f"\nTotal DOR (incl. matched KPOCT, excl. DOR-only KPOCT/Pedi-Onc): {total_dor_effective:,.2f}"
# )
# print(f"Total VP (Transaction Amount): {total_vp:,.2f}")
# print(f"Overlap (both datasets): {overlap:,.2f}")
# print(f"DOR only (not in VP; excl. KPOCT/Pedi-Onc): {dor_only_total:,.2f}")
# print(
#     f"VP only (not in DOR): {merged.loc[merged['_merge']=='right_only', 'Transaction Amount'].sum():,.2f}"
# )
# dor_only_mask = (merged["_merge"] == "left_only") & (
#     ~merged["Program Area"].isin(["KPOCT", "Pedi-Onc"])
# )
# dor_only_total = merged.loc[dor_only_mask, "Total Cash Receipts"].sum()
# print(f"DOR minus overlap (excl. KPOCT/Pedi-Onc): {dor_only_total:,.2f}")
# print(f"DOR minus overlap: {dor_only_total:,.2f}")
# print(f"VP minus overlap: {total_vp - overlap:,.2f}")

In [ ]:
import pandas as pd
from datetime import datetime

# =========================
# CONFIGURATION
# =========================

# Date range
START_DATE = "2000-01-01"
END_DATE = "2025-12-31"

# Input file paths
DOR_PATH = (
    r"C:\Users\O304312\OneDrive - Kaiser Permanente\Documents\DOR Transactions.xlsx"
)
VP_PATH = r"C:\Users\O304312\Downloads\Transaction.xlsx"

# Column names used in each system
DOR_DATE_COL = "Award Term Start Date"
DOR_ID_COL = "Project ID"
DOR_TITLE_COL = "Project Title"
DOR_AMOUNT_COL = "Total Cash Receipts"
DOR_PROGRAM_COL = "Program Area"

VP_DATE_COL = "Accountable Completed Date"
VP_ID_COL = "Service Line Code"  # This is what should match Project ID
VP_AMOUNT_COL = "Transaction Amount"
VP_STUDY_CODE_COL = "Site Study Code"

# Program areas to exclude **only** when they are DOR-only
EXCLUDE_DOR_ONLY_PROGRAMS = ["KPOCT", "Pedi-Onc"]

# Output files
MERGED_OUT = "recon_merged.csv"
DOR_ONLY_OUT = "dor_only_ids.csv"
VP_ONLY_OUT = "vp_only_ids.csv"

# =========================
# HELPER FUNCTIONS
# =========================


def load_and_clean_dor(path: str, start_date: str, end_date: str) -> pd.DataFrame:
    """Load DOR data, filter by date, and keep key columns."""
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    dor = pd.read_excel(path)

    # Make sure required columns exist
    for col in [
        DOR_DATE_COL,
        DOR_ID_COL,
        DOR_TITLE_COL,
        DOR_AMOUNT_COL,
        DOR_PROGRAM_COL,
    ]:
        if col not in dor.columns:
            dor[col] = pd.NA

    # Date filter
    dor[DOR_DATE_COL] = pd.to_datetime(dor[DOR_DATE_COL], errors="coerce")
    dor = dor[(dor[DOR_DATE_COL] >= start) & (dor[DOR_DATE_COL] <= end)]

    print(f"DOR rows after date filter: {len(dor)}")

    # Keep only the columns we actually need
    dor = dor[[DOR_ID_COL, DOR_TITLE_COL, DOR_AMOUNT_COL, DOR_PROGRAM_COL]].copy()

    # Clean amount
    dor[DOR_AMOUNT_COL] = pd.to_numeric(dor[DOR_AMOUNT_COL], errors="coerce").fillna(0)

    return dor


def load_and_clean_vp(path: str, start_date: str, end_date: str) -> pd.DataFrame:
    """Load VP data, dedupe, filter by date, and aggregate by ID."""
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    vp = pd.read_excel(path)

    # Drop technical index column if present
    if "Unnamed: 0" in vp.columns:
        vp = vp.drop(columns=["Unnamed: 0"])

    before_dups = len(vp)
    vp = vp.drop_duplicates()
    print(f"VP duplicate rows removed: {before_dups - len(vp)}")

    # Ensure columns exist
    for col in [VP_DATE_COL, VP_ID_COL, VP_AMOUNT_COL, VP_STUDY_CODE_COL]:
        if col not in vp.columns:
            vp[col] = pd.NA

    # Date filter
    vp[VP_DATE_COL] = pd.to_datetime(vp[VP_DATE_COL], errors="coerce")
    vp = vp[(vp[VP_DATE_COL] >= start) & (vp[VP_DATE_COL] <= end)]
    print(f"VP rows after date filter: {len(vp)}")

    # Clean amount
    vp[VP_AMOUNT_COL] = pd.to_numeric(vp[VP_AMOUNT_COL], errors="coerce").fillna(0)

    # Drop rows with no ID
    vp = vp.dropna(subset=[VP_ID_COL])
    vp_detail = vp.copy()

    # Aggregate by ID and collect unique Site Study Codes
    vp_grouped = vp.groupby(VP_ID_COL, as_index=False).agg(
        {
            VP_AMOUNT_COL: "sum",
            VP_STUDY_CODE_COL: lambda s: (
                "; ".join(sorted(set(map(str, s.dropna()))))
                if s.notna().any()
                else pd.NA
            ),
        }
    )

    return vp_grouped


def merge_and_reconcile(dor: pd.DataFrame, vp: pd.DataFrame) -> pd.DataFrame:
    """Outer-join DOR and VP and compute reconciliation fields."""
    merged = dor.merge(
        vp,
        how="outer",
        left_on=DOR_ID_COL,
        right_on=VP_ID_COL,
        indicator=True,
        suffixes=("_dor", "_vp"),
    )

    # Standardize amounts and fill missing with 0
    merged[DOR_AMOUNT_COL] = merged[DOR_AMOUNT_COL].fillna(0)
    merged[VP_AMOUNT_COL] = merged[VP_AMOUNT_COL].fillna(0)

    # Difference (DOR minus VP)
    merged["Difference"] = merged[DOR_AMOUNT_COL] - merged[VP_AMOUNT_COL]

    return merged


# =========================
# MAIN LOGIC
# =========================

if __name__ == "__main__":
    print(f"Filtering data from {START_DATE} to {END_DATE}\n")

    # Load and clean each source
    dor = load_and_clean_dor(DOR_PATH, START_DATE, END_DATE)
    vp = load_and_clean_vp(VP_PATH, START_DATE, END_DATE)

    # Merge
    merged = merge_and_reconcile(dor, vp)

    # Masks for match status
    matched_mask = merged["_merge"] == "both"
    dor_only_mask = merged["_merge"] == "left_only"
    vp_only_mask = merged["_merge"] == "right_only"

    # DOR-only but exclude certain Program Areas
    dor_only_effective_mask = dor_only_mask & (
        ~merged[DOR_PROGRAM_COL].isin(EXCLUDE_DOR_ONLY_PROGRAMS)
    )

    # Lists for review
    dor_only_ids = merged.loc[dor_only_effective_mask, DOR_ID_COL].dropna().tolist()
    vp_only_ids = merged.loc[vp_only_mask, VP_ID_COL].dropna().tolist()

    print(
        f"{len(dor_only_ids)} DOR Project IDs not in VP (excluding {EXCLUDE_DOR_ONLY_PROGRAMS}):"
    )
    print(dor_only_ids)

    print(f"\n{len(vp_only_ids)} VP Service Line Codes not in DOR:")
    print(vp_only_ids)

    # Totals and overlap
    total_dor = dor[DOR_AMOUNT_COL].sum()
    total_vp = vp[VP_AMOUNT_COL].sum()

    overlap_dor = merged.loc[matched_mask, DOR_AMOUNT_COL].sum()
    overlap_vp = merged.loc[matched_mask, VP_AMOUNT_COL].sum()

    dor_only_total_effective = merged.loc[dor_only_effective_mask, DOR_AMOUNT_COL].sum()
    vp_only_total = merged.loc[vp_only_mask, VP_AMOUNT_COL].sum()

    print("\n===== SUMMARY =====")
    print(f"Total DOR (within date range): {total_dor:,.2f}")
    print(f"Total VP  (within date range): {total_vp:,.2f}\n")

    print(f"Matched DOR amount (overlap): {overlap_dor:,.2f}")
    print(f"Matched VP  amount (overlap): {overlap_vp:,.2f}")
    print(
        f"Difference on matched records (DOR - VP): {(overlap_dor - overlap_vp):,.2f}\n"
    )

    print(
        f"DOR-only (excl. {EXCLUDE_DOR_ONLY_PROGRAMS}): {dor_only_total_effective:,.2f}"
    )
    print(f"VP-only: {vp_only_total:,.2f}")
    print(
        f"Check: (DOR total - overlap_dor - excluded DOR-only) vs (VP total - overlap_vp - VP-only)"
    )
    print("This can help you see if the reconciliation is balanced under your rules.\n")

    # =========================
    # EXPORTS
    # =========================

    # Full merged reconciliation table
    merged.to_csv(MERGED_OUT, index=False)

    # DOR-only export
    merged.loc[
        dor_only_effective_mask,
        [DOR_ID_COL, DOR_TITLE_COL, DOR_AMOUNT_COL, DOR_PROGRAM_COL],
    ].to_csv(DOR_ONLY_OUT, index=False)

    # VP-only export
    merged.loc[
        vp_only_mask,
        [VP_ID_COL, VP_STUDY_CODE_COL, VP_AMOUNT_COL],
    ].to_csv(VP_ONLY_OUT, index=False)

    print(
        f"Exported '{MERGED_OUT}', '{DOR_ONLY_OUT}', and '{VP_ONLY_OUT}' for detailed review."
    )

Filtering data from 2000-01-01 to 2025-12-31

DOR rows after date filter: 194
VP duplicate rows removed: 1803
VP rows after date filter: 28185
42 DOR Project IDs not in VP (excluding ['KPOCT', 'Pedi-Onc']):
['RNG021526', 'RNG200328', 'RNG200517', 'RNG200521', 'RNG200603', 'RNG200644', 'RNG209611', 'RNG209693', 'RNG210008', 'RNG210130', 'RNG210199', 'RNG210298', 'RNG210619', 'RNG210686', 'RNG210695', 'RNG210994', 'RNG211179', 'RNG211195', 'RNG211207', 'RNG211382', 'RNG211537', 'RNG211611', 'RNG211930', 'RNG211939', 'RNG212007', 'RNG212008', 'RNG212074', 'RNG212237', 'RNG212367', 'RNG212651', 'RNG212684', 'RNG212710', 'RNG212772', 'RNG213027', 'RNG213030', 'RNG213253', 'RNG213695', 'RNG213696', 'RNG213698', 'RNG213786', 'RNG213836', 'RNG213869']

3 VP Service Line Codes not in DOR:
['RNG200066', 'RNG200486', 'RNG212075']

===== SUMMARY =====
Total DOR (within date range): 55,729,786.61
Total VP  (within date range): 20,746,834.81

Matched DOR amount (overlap): 19,427,569.65
Matched VP  a

In [ ]:
import pandas as pd

# -----------------------------
# Build a "matched" studies table
# -----------------------------
matched_mask = merged["_merge"] == "both"
matched = merged.loc[matched_mask].copy()

# Add a difference column (DOR - VP)
matched["Difference"] = matched["Total Cash Receipts"] - matched["Transaction Amount"]

# Keep just the key fields for the report
matched_report = matched[
    [
        "Project ID",
        "Project Title",
        "Program Area",
        "Total Cash Receipts",
        "Transaction Amount",
        "Difference",
    ]
].sort_values("Project ID")

# -----------------------------
# Summary table using existing numbers
# -----------------------------
summary_rows = [
    {
        "Metric": "Total DOR (effective rule)",
        "Value": total_dor_effective,
    },
    {
        "Metric": "Total VP",
        "Value": total_vp,
    },
    {
        "Metric": "Overlap (DOR side)",
        "Value": overlap,
    },
    {
        "Metric": "DOR-only (excl. KPOCT/Pedi-Onc)",
        "Value": dor_only_total,
    },
    {
        "Metric": "VP-only",
        "Value": merged.loc[
            merged["_merge"] == "right_only", "Transaction Amount"
        ].sum(),
    },
    {
        "Metric": "VP minus overlap",
        "Value": total_vp - overlap,
    },
]

summary_df = pd.DataFrame(summary_rows)

# -----------------------------
# Prepare DOR and VP detail tables
# -----------------------------
dor_detail = dor.copy()

# For VP detail, keep the key columns
vp_detail_report = vp_detail[
    [
        "Service Line Code",
        "Site Study Code",
        "Accountable Completed Date",
        "Transaction Amount",
    ]
].copy()

# -----------------------------
# Write to Excel with interactivity
# -----------------------------
output_file = "reconciliation_report.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # 1. Summary
    summary_df.to_excel(writer, sheet_name="Summary", index=False)

    # 2. Matched studies (for overview + dropdown)
    matched_report.to_excel(writer, sheet_name="Matched_Studies", index=False)

    # 3. DOR detail
    dor_detail.to_excel(writer, sheet_name="DOR_Detail", index=False)

    # 4. VP detail (all transactions)
    vp_detail_report.to_excel(writer, sheet_name="VP_Detail", index=False)

    workbook = writer.book

    # 5. Interactive detail sheet
    detail_sheet = workbook.add_worksheet("Study_Detail")
    writer.sheets["Study_Detail"] = detail_sheet

    # --- Dropdown setup ---
    detail_sheet.write("A1", "Select Project ID:")
    detail_sheet.write("B1", "")

    # Matched_Studies has Project ID in column A starting at row 2
    matched_n = len(matched_report)
    # e.g., =Matched_Studies!$A$2:$A$101
    dv_range = f"=Matched_Studies!$A$2:$A${matched_n + 1}"

    detail_sheet.data_validation(
        "B1",
        {
            "validate": "list",
            "source": dv_range,
        },
    )

    # Little labels
    detail_sheet.write("A3", "DOR Detail for selected Project ID")
    detail_sheet.write("A10", "VP Transactions for selected Project ID")

    # Helper to convert 0-based column index to Excel column letters (A, B, ..., AA, AB, ...)
    def colnum_to_excel_col(col_num: int) -> str:
        col = ""
        n = col_num + 1
        while n > 0:
            n, remainder = divmod(n - 1, 26)
            col = chr(65 + remainder) + col
        return col

    # --- DOR detail FILTER formula ---
    # dor_detail was written to DOR_Detail, columns start at A1, headers in row 1, data in row 2+
    dor_cols = list(dor_detail.columns)
    dor_last_col_letter = colnum_to_excel_col(len(dor_cols) - 1)  # e.g. 'D'
    dor_data_range = f"DOR_Detail!A2:{dor_last_col_letter}1048576"  # large range
    dor_id_col = "A"  # assuming Project ID is first col in dor_detail

    # Write headers for DOR detail table
    for c, col_name in enumerate(dor_cols):
        detail_sheet.write(4, c, col_name)  # row 5 (0-based index 4)

    # Dynamic FILTER formula for DOR detail (spills)
    dor_filter_formula = (
        f"=FILTER({dor_data_range}, DOR_Detail!{dor_id_col}2:{dor_id_col}1048576=$B$1)"
    )
    detail_sheet.write_formula(5, 0, dor_filter_formula)  # row 6, col A

    # --- VP detail FILTER formula ---
    vp_cols = list(vp_detail_report.columns)
    vp_last_col_letter = colnum_to_excel_col(len(vp_cols) - 1)  # e.g. 'D'
    vp_data_range = f"VP_Detail!A2:{vp_last_col_letter}1048576"
    vp_id_col = "A"  # assuming Service Line Code is first col in vp_detail_report

    # Write headers for VP detail table
    for c, col_name in enumerate(vp_cols):
        detail_sheet.write(11, c, col_name)  # row 12 (0-based index 11)

    # FILTER VP rows where Service Line Code == selected Project ID
    vp_filter_formula = (
        f"=FILTER({vp_data_range}, VP_Detail!{vp_id_col}2:{vp_id_col}1048576=$B$1)"
    )
    detail_sheet.write_formula(12, 0, vp_filter_formula)  # row 13, col A

print(f"\nExcel report written to: {output_file}")

NameError: name 'total_dor_effective' is not defined

In [ ]:
dor = dor.merge(vp, how="left", left_on="Project ID", right_on="Service Line Code")
dor["Transaction Amount"] = dor["Transaction Amount"].fillna(0)
dor["diff"] = dor["Total Cash Receipts"] - dor["Transaction Amount"]
dor["diff"] = dor["diff"].round(2)
print(dor["diff"].sum())

35215291.86
